In [1]:
import pandas as pd #table organization
import numpy as np #math operations
import re #regex

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200) #just for visualization

In [2]:
from google.colab import files
uploaded = files.upload()

Saving rep_s_00014_SMRY.csv to rep_s_00014_SMRY (2).csv
Saving REP_S_00134_SMRY.csv to REP_S_00134_SMRY (2).csv
Saving rep_s_00191_SMRY-3.csv to rep_s_00191_SMRY-3 (2).csv
Saving rep_s_00673_SMRY.csv to rep_s_00673_SMRY (2).csv


In [3]:
monthly = pd.read_csv("REP_S_00134_SMRY.csv", dtype=str) #str so that we just get the raw data
product = pd.read_csv("rep_s_00014_SMRY.csv", dtype=str)
category = pd.read_csv("rep_s_00673_SMRY.csv", dtype=str)
group = pd.read_csv("rep_s_00191_SMRY-3.csv", dtype=str)

monthly.head() #inspecting the data

,Stories,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Comparative Monthly Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22-Jan-2026,NaN,"Year: 2026,2025",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Page 1 of,0.01
2,NaN,NaN,NaN,January,February,March,April,May,June,July,August,September,NaN,NaN
3,2025,Stories - Bir Hasan,NaN,"3,355,705.33","2,842,993.67","2,266,050.67","3,459,979.67","2,125,379.33","744,638.33","3,799,740.25","3,783,897.91","3,255,935.75",NaN,NaN
4,NaN,Stories Ain El Mreisseh,NaN,"12,648,546.67","9,929,973.33","9,849,926.67","13,025,923.33","7,208,975.00","2,883,403.33","11,889,329.16","11,610,097.92","11,021,123.75",NaN,NaN


In [4]:
mon0 = pd.read_csv("REP_S_00134_SMRY.csv", header=None, dtype=str)
headerIdx = mon0.index[mon0.apply(lambda r: r.astype(str).str.contains("January", na=False).any(), axis=1)][0]
headerIdx
#now we use obtained row as our header
monthlyClean = mon0.iloc[headerIdx:].copy()
monthlyClean.columns = monthlyClean.iloc[0]
monthlyClean = monthlyClean.iloc[1:].reset_index(drop=True)

In [5]:
rowText = monthlyClean.astype(str).agg(" ".join, axis=1).str.lower()
monthlyClean = monthlyClean[~rowText.str.contains(r"\bjanuary\b", na=False)].reset_index(drop=True)

monthlyClean.head()

3,NaN,NaN,NaN,January,February,March,April,May,June,July,August,September,NaN,NaN
0,2025,Stories - Bir Hasan,NaN,"3,355,705.33","2,842,993.67","2,266,050.67","3,459,979.67","2,125,379.33","744,638.33","3,799,740.25","3,783,897.91","3,255,935.75",NaN,NaN
1,NaN,Stories Ain El Mreisseh,NaN,"12,648,546.67","9,929,973.33","9,849,926.67","13,025,923.33","7,208,975.00","2,883,403.33","11,889,329.16","11,610,097.92","11,021,123.75",NaN,NaN
2,NaN,Stories Airport,NaN,0.00,0.00,0.00,0.00,0.00,"8,428.83","2,940,003.01","6,963,426.45","7,467,197.62",NaN,NaN
3,NaN,Stories Antelias,NaN,"2,615,854.33","2,139,010.67","3,162,716.67","3,391,741.33","2,033,635.67","728,978.33","2,963,696.71","3,182,363.97","2,909,965.47",NaN,NaN
4,NaN,Stories Batroun,NaN,"4,266,517.33","3,388,117.33","4,890,198.33","5,516,881.67","2,963,980.33","1,311,240.00","6,135,003.02","6,751,770.29","5,041,150.76",NaN,NaN


In [6]:
monthlyClean["Year"] = monthlyClean.iloc[:, 0]
monthlyClean["Branch Name"] = monthlyClean.iloc[:, 1]

monthlyClean[["Year", "Branch Name", "January", "February"]].head(10)

3,Year,Branch Name,January,February
0,2025,Stories - Bir Hasan,"3,355,705.33","2,842,993.67"
1,NaN,Stories Ain El Mreisseh,"12,648,546.67","9,929,973.33"
2,NaN,Stories Airport,0.00,0.00
3,NaN,Stories Antelias,"2,615,854.33","2,139,010.67"
4,NaN,Stories Batroun,"4,266,517.33","3,388,117.33"
5,NaN,Stories Bayada,"4,497,376.67","3,419,106.67"
6,NaN,Stories Centro Mall,"3,264,533.33","2,944,806.67"
7,NaN,Stories Event Starco,0.00,0.00
8,NaN,Stories Faqra,"1,889,910.00","1,829,960.00"
9,NaN,Stories Khaldeh,"7,468,155.33","6,108,429.33"


In [7]:
#We want to forward fill the year for clarity. we have filler columns (Branch name).
monthlyClean["Year"] = monthlyClean["Year"].ffill()
monthlyClean[["Year", "Branch Name"]].head(15)

3,Year,Branch Name
0,2025,Stories - Bir Hasan
1,2025,Stories Ain El Mreisseh
2,2025,Stories Airport
3,2025,Stories Antelias
4,2025,Stories Batroun
5,2025,Stories Bayada
6,2025,Stories Centro Mall
7,2025,Stories Event Starco
8,2025,Stories Faqra
9,2025,Stories Khaldeh


In [8]:
#just for verification
monthlyClean["Year"].value_counts(dropna=False)

,count
Year,
2025,54
2026,51


In [9]:
bn = monthlyClean.loc[:, "Branch Name"]
branch1 = monthlyClean.iloc[:, monthlyClean.columns.get_loc("Branch Name")]
# removing duplicate branch names
branch1 = monthlyClean.loc[:, "Branch Name"]
if isinstance(branch1, pd.DataFrame):
    branch1 = branch1.iloc[:, 0]  # keep only first if duplicates exist

monthCols = [c for c in ["January","February","March","April","May","June","July","August","September","October","November","December"]
             if c in monthlyClean.columns]

monthlyClean = pd.concat([monthlyClean["Year"].iloc[:, 0] if isinstance(monthlyClean["Year"], pd.DataFrame) else monthlyClean["Year"],
                          branch1.rename("Branch Name"),
                          monthlyClean[monthCols]], axis=1)

monthlyClean.head()

,Year,Branch Name,January,February,March,April,May,June,July,August,September
0,2025,Stories - Bir Hasan,"3,355,705.33","2,842,993.67","2,266,050.67","3,459,979.67","2,125,379.33","744,638.33","3,799,740.25","3,783,897.91","3,255,935.75"
1,2025,Stories Ain El Mreisseh,"12,648,546.67","9,929,973.33","9,849,926.67","13,025,923.33","7,208,975.00","2,883,403.33","11,889,329.16","11,610,097.92","11,021,123.75"
2,2025,Stories Airport,0.00,0.00,0.00,0.00,0.00,"8,428.83","2,940,003.01","6,963,426.45","7,467,197.62"
3,2025,Stories Antelias,"2,615,854.33","2,139,010.67","3,162,716.67","3,391,741.33","2,033,635.67","728,978.33","2,963,696.71","3,182,363.97","2,909,965.47"
4,2025,Stories Batroun,"4,266,517.33","3,388,117.33","4,890,198.33","5,516,881.67","2,963,980.33","1,311,240.00","6,135,003.02","6,751,770.29","5,041,150.76"


In [10]:
#just noticed that october is not with them, it is placed elsehwere with nov and dec. I'm bringing them all together.
octHeaderIdx = mon0.index[
    mon0.apply(lambda r: r.astype(str).str.contains("October", na=False).any(), axis=1)
][0]
octHeaderIdx

mon0.iloc[octHeaderIdx:octHeaderIdx+6, :15]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
57,NaN,NaN,October,November,December,Total By Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2025,Stories - Bir Hasan,"3,128,528.54","2,851,287.10","2,743,087.70","34,357,224.24",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,NaN,Stories Ain El Mreisseh,"10,516,783.81","9,387,782.61","9,640,996.43","119,612,862.02",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,October,November,December,Total By Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,2025,Stories Airport,"8,414,672.70","7,029,720.14","6,604,828.86","39,428,277.60",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,Stories Antelias,"2,715,152.86","2,551,353.16","2,234,759.77","30,629,228.94",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
oct0 = mon0.iloc[octHeaderIdx:].copy()
oct0 = oct0.iloc[:, :6]

oct0.head(10)

,0,1,2,3,4,5
57,NaN,NaN,October,November,December,Total By Year
58,2025,Stories - Bir Hasan,"3,128,528.54","2,851,287.10","2,743,087.70","34,357,224.24"
59,NaN,Stories Ain El Mreisseh,"10,516,783.81","9,387,782.61","9,640,996.43","119,612,862.02"
60,NaN,NaN,October,November,December,Total By Year
61,2025,Stories Airport,"8,414,672.70","7,029,720.14","6,604,828.86","39,428,277.60"
62,NaN,Stories Antelias,"2,715,152.86","2,551,353.16","2,234,759.77","30,629,228.94"
63,NaN,Stories Batroun,"4,372,225.54","4,272,667.88","5,163,712.03","54,073,464.51"
64,NaN,Stories Bayada,"5,224,336.34","4,588,470.28","4,403,458.27","52,643,538.48"
65,NaN,Stories Centro Mall,"3,059,645.65","3,497,554.96","3,998,663.08","37,541,717.86"
66,NaN,Stories Event Starco,0.00,"6,606.61","586,645.65","597,312.31"


In [12]:
octHeader = oct0.iloc[0].tolist()
oct = oct0.iloc[1:].copy()
oct.columns = ["Year", "Branch Name", "October", "November", "December", "Total By Year"]
oct = oct[oct["October"].astype(str).str.strip().str.lower() != "october"].reset_index(drop=True)

oct.head(10)

,Year,Branch Name,October,November,December,Total By Year
0,2025,Stories - Bir Hasan,"3,128,528.54","2,851,287.10","2,743,087.70","34,357,224.24"
1,NaN,Stories Ain El Mreisseh,"10,516,783.81","9,387,782.61","9,640,996.43","119,612,862.02"
2,2025,Stories Airport,"8,414,672.70","7,029,720.14","6,604,828.86","39,428,277.60"
3,NaN,Stories Antelias,"2,715,152.86","2,551,353.16","2,234,759.77","30,629,228.94"
4,NaN,Stories Batroun,"4,372,225.54","4,272,667.88","5,163,712.03","54,073,464.51"
5,NaN,Stories Bayada,"5,224,336.34","4,588,470.28","4,403,458.27","52,643,538.48"
6,NaN,Stories Centro Mall,"3,059,645.65","3,497,554.96","3,998,663.08","37,541,717.86"
7,NaN,Stories Event Starco,0.00,"6,606.61","586,645.65","597,312.31"
8,NaN,Stories Faqra,0.00,0.00,0.00,"10,977,393.51"
9,NaN,Stories Khaldeh,"7,348,415.03","6,744,352.33","6,874,042.67","84,155,611.99"


In [13]:
oct["Year"] = oct["Year"].ffill()
oct[["Year", "Branch Name", "October", "Total By Year"]].head(12)

,Year,Branch Name,October,Total By Year
0,2025,Stories - Bir Hasan,"3,128,528.54","34,357,224.24"
1,2025,Stories Ain El Mreisseh,"10,516,783.81","119,612,862.02"
2,2025,Stories Airport,"8,414,672.70","39,428,277.60"
3,2025,Stories Antelias,"2,715,152.86","30,629,228.94"
4,2025,Stories Batroun,"4,372,225.54","54,073,464.51"
5,2025,Stories Bayada,"5,224,336.34","52,643,538.48"
6,2025,Stories Centro Mall,"3,059,645.65","37,541,717.86"
7,2025,Stories Event Starco,0.00,"597,312.31"
8,2025,Stories Faqra,0.00,"10,977,393.51"
9,2025,Stories Khaldeh,"7,348,415.03","84,155,611.99"


In [14]:
#keeping only needed Oct–Dec columns
octMerge = oct[["Year", "Branch Name", "October", "November", "December"]].copy()
#merging on year and branch name (plsplspls work)
monthlyClean = monthlyClean.merge(
    octMerge,
    on=["Year", "Branch Name"],
    how="left"
)

monthlyClean.head()

,Year,Branch Name,January,February,March,April,May,June,July,August,September,October,November,December
0,2025,Stories - Bir Hasan,"3,355,705.33","2,842,993.67","2,266,050.67","3,459,979.67","2,125,379.33","744,638.33","3,799,740.25","3,783,897.91","3,255,935.75","3,128,528.54","2,851,287.10","2,743,087.70"
1,2025,Stories Ain El Mreisseh,"12,648,546.67","9,929,973.33","9,849,926.67","13,025,923.33","7,208,975.00","2,883,403.33","11,889,329.16","11,610,097.92","11,021,123.75","10,516,783.81","9,387,782.61","9,640,996.43"
2,2025,Stories Airport,0.00,0.00,0.00,0.00,0.00,"8,428.83","2,940,003.01","6,963,426.45","7,467,197.62","8,414,672.70","7,029,720.14","6,604,828.86"
3,2025,Stories Antelias,"2,615,854.33","2,139,010.67","3,162,716.67","3,391,741.33","2,033,635.67","728,978.33","2,963,696.71","3,182,363.97","2,909,965.47","2,715,152.86","2,551,353.16","2,234,759.77"
4,2025,Stories Batroun,"4,266,517.33","3,388,117.33","4,890,198.33","5,516,881.67","2,963,980.33","1,311,240.00","6,135,003.02","6,751,770.29","5,041,150.76","4,372,225.54","4,272,667.88","5,163,712.03"


In [15]:
#https://pbpython.com/currency-cleanup.html
def toNumber(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(",", "").strip()
    s = re.sub(r"[^0-9\.\-]", "", s)
    if s == "":
        return np.nan
    return float(s)

monthCols = ["January","February","March","April","May","June","July","August","September","October","November","December"]

for c in monthCols:
    monthlyClean[c] = monthlyClean[c].map(toNumber)

monthlyClean.dtypes

,0
Year,object
Branch Name,object
January,float64
February,float64
March,float64
April,float64
May,float64
June,float64
July,float64
August,float64


In [16]:
monthlyClean.head()

,Year,Branch Name,January,February,March,April,May,June,July,August,September,October,November,December
0,2025,Stories - Bir Hasan,3355705.33,2842993.67,2266050.67,3459979.67,2125379.33,744638.33,3799740.25,3783897.91,3255935.75,3128528.54,2851287.10,2743087.70
1,2025,Stories Ain El Mreisseh,12648546.67,9929973.33,9849926.67,13025923.33,7208975.00,2883403.33,11889329.16,11610097.92,11021123.75,10516783.81,9387782.61,9640996.43
2,2025,Stories Airport,0.00,0.00,0.00,0.00,0.00,8428.83,2940003.01,6963426.45,7467197.62,8414672.70,7029720.14,6604828.86
3,2025,Stories Antelias,2615854.33,2139010.67,3162716.67,3391741.33,2033635.67,728978.33,2963696.71,3182363.97,2909965.47,2715152.86,2551353.16,2234759.77
4,2025,Stories Batroun,4266517.33,3388117.33,4890198.33,5516881.67,2963980.33,1311240.00,6135003.02,6751770.29,5041150.76,4372225.54,4272667.88,5163712.03


In [17]:
#annual total
monthlyClean["Annual Total"] = monthlyClean[monthCols].sum(axis=1)
pd.options.display.float_format = '{:,.2f}'.format
monthlyClean[["Year","Branch Name","Annual Total"]].head()

,Year,Branch Name,Annual Total
0,2025,Stories - Bir Hasan,"34,357,224.25"
1,2025,Stories Ain El Mreisseh,"119,612,862.01"
2,2025,Stories Airport,"39,428,277.61"
3,2025,Stories Antelias,"30,629,228.94"
4,2025,Stories Batroun,"54,073,464.51"


In [18]:
prod0 = pd.read_csv("rep_s_00014_SMRY.csv", header=None, dtype=str)
prodHeaderIdx = prod0.index[
    prod0.apply(lambda r: r.astype(str).str.contains("Product Desc", na=False).any(), axis=1)
][0]
prodHeaderIdx

np.int64(3)

In [19]:
prod = prod0.iloc[prodHeaderIdx+1:].copy()

prod.columns = [
    "Product Desc",
    "Qty",
    "Total Price",
    "Blank1",
    "Total Cost",
    "Total Cost %",
    "Total Profit",
    "Blank2",
    "Total Profit %",
    "Blank3"
]

prod.head(15)

,Product Desc,Qty,Total Price,Blank1,Total Cost,Total Cost %,Total Profit,Blank2,Total Profit %,Blank3
4,Stories - Bir Hasan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TAKE AWAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BEVERAGES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,COLD BAR SECTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1 SHOT DECAFE,404.00,0.00,NaN,"3,856.85",0.00,"-3,856.85",NaN,100.00,NaN
9,2 SHOT DECAFE,637.00,0.00,NaN,"12,162.45",0.00,"-12,162.45",NaN,100.00,NaN
10,3 SHOT DECAFE,113.00,0.00,NaN,"3,236.32",0.00,"-3,236.32",NaN,100.00,NaN
11,ADD BANANA SAUCE LARGE,4.00,266.67,NaN,139.97,52.49,126.69,NaN,47.51,NaN
12,ADD BANANA SAUCE MEDIUM,47.00,"1,487.39",NaN,822.34,55.29,665.04,NaN,44.71,NaN
13,ADD BANANA SAUCE SMALL,28.00,893.72,NaN,489.91,54.82,403.82,NaN,45.18,NaN


In [20]:
#tagging the row
isQty = prod["Qty"].notna()
desc = prod["Product Desc"].astype(str).str.strip()

isBranch = desc.str.startswith("Stories")
isService = desc.isin(["TAKE AWAY", "TABLE"])
isCategory = desc.isin(["BEVERAGES", "FOOD"])
is_section = (~isQty) & (~isBranch) & (~isService) & (~isCategory) & desc.ne("nan") & desc.ne("")

isBranch.sum(), isService.sum(), isCategory.sum(), is_section.sum(), isQty.sum()

(np.int64(25), np.int64(35), np.int64(80), np.int64(902), np.int64(13539))

In [21]:
prod["Branch"] = None
prod["Service Type"] = None
prod["Category"] = None
prod["Section"] = None

prod.loc[isBranch, "Branch"] = prod.loc[isBranch, "Product Desc"]
prod.loc[isService, "Service Type"] = prod.loc[isService, "Product Desc"]
prod.loc[isCategory, "Category"] = prod.loc[isCategory, "Product Desc"]
prod.loc[is_section, "Section"] = prod.loc[is_section, "Product Desc"]

prod["Branch"] = prod["Branch"].ffill()
prod["Service Type"] = prod["Service Type"].ffill()
prod["Category"] = prod["Category"].ffill()
prod["Section"] = prod["Section"].ffill()

prod.head(20)

,Product Desc,Qty,Total Price,Blank1,Total Cost,Total Cost %,Total Profit,Blank2,Total Profit %,Blank3,Branch,Service Type,Category,Section
4,Stories - Bir Hasan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stories - Bir Hasan,None,None,None
5,TAKE AWAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stories - Bir Hasan,TAKE AWAY,None,None
6,BEVERAGES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,None
7,COLD BAR SECTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
8,1 SHOT DECAFE,404.00,0.00,NaN,"3,856.85",0.00,"-3,856.85",NaN,100.00,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
9,2 SHOT DECAFE,637.00,0.00,NaN,"12,162.45",0.00,"-12,162.45",NaN,100.00,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
10,3 SHOT DECAFE,113.00,0.00,NaN,"3,236.32",0.00,"-3,236.32",NaN,100.00,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
11,ADD BANANA SAUCE LARGE,4.00,266.67,NaN,139.97,52.49,126.69,NaN,47.51,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
12,ADD BANANA SAUCE MEDIUM,47.00,"1,487.39",NaN,822.34,55.29,665.04,NaN,44.71,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
13,ADD BANANA SAUCE SMALL,28.00,893.72,NaN,489.91,54.82,403.82,NaN,45.18,NaN,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION


In [22]:
prodItems = prod[isQty].copy()
prodItems.shape, prodItems.head(5)

((13539, 14),
                Product Desc     Qty Total Price Blank1 Total Cost Total Cost % Total Profit Blank2 Total Profit % Blank3               Branch Service Type   Category           Section
 8             1 SHOT DECAFE  404.00        0.00    NaN   3,856.85         0.00    -3,856.85    NaN         100.00    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 9             2 SHOT DECAFE  637.00        0.00    NaN  12,162.45         0.00   -12,162.45    NaN         100.00    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 10            3 SHOT DECAFE  113.00        0.00    NaN   3,236.32         0.00    -3,236.32    NaN         100.00    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 11   ADD BANANA SAUCE LARGE    4.00      266.67    NaN     139.97        52.49       126.69    NaN          47.51    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 12  ADD BANANA SAUCE MEDIUM   47.00    1,487.39    NaN     8

In [23]:
bad = {}

for c in ["Qty", "Total Price", "Total Cost", "Total Cost %", "Total Profit", "Total Profit %"]:
    s = prodItems[c].astype(str)
    mask = s.str.contains(r"[A-Za-z]", na=False) | s.str.contains(r"\.\.", na=False)
    bad[c] = prodItems.loc[mask, ["Product Desc", c, "Branch", "Service Type", "Category", "Section"]].head(10)

bad

{'Qty':      Product Desc  Qty               Branch Service Type   Category    Section
 38   Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 75   Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 112  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 148  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 184  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 220  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 257  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  22-Jan-26
 294  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY       FOOD  22-Jan-26
 331  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY       FOOD  22-Jan-26
 367  Product Desc  Qty  Stories - Bir Hasan    TAKE AWAY       FOOD  22-Jan-26,
 'Total Price':      Product Desc  Total Price               Branch Service Type   Category    Section
 38   Pro

In [24]:
#drop rows where the Qty column is js the word Qty
prodItems = prodItems[prodItems["Qty"].astype(str).str.strip().str.lower() != "qty"].copy()

prodItems.shape, prodItems.head(5)

((13144, 14),
                Product Desc     Qty Total Price Blank1 Total Cost Total Cost % Total Profit Blank2 Total Profit % Blank3               Branch Service Type   Category           Section
 8             1 SHOT DECAFE  404.00        0.00    NaN   3,856.85         0.00    -3,856.85    NaN         100.00    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 9             2 SHOT DECAFE  637.00        0.00    NaN  12,162.45         0.00   -12,162.45    NaN         100.00    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 10            3 SHOT DECAFE  113.00        0.00    NaN   3,236.32         0.00    -3,236.32    NaN         100.00    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 11   ADD BANANA SAUCE LARGE    4.00      266.67    NaN     139.97        52.49       126.69    NaN          47.51    NaN  Stories - Bir Hasan    TAKE AWAY  BEVERAGES  COLD BAR SECTION
 12  ADD BANANA SAUCE MEDIUM   47.00    1,487.39    NaN     8

In [25]:
def toNumber(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(",", "").strip()
    s = re.sub(r"[^0-9\.\-]", "", s)
    if s == "":
        return np.nan
    return float(s)

In [26]:
numCols = [
    "Qty",
    "Total Price",
    "Total Cost",
    "Total Cost %",
    "Total Profit",
    "Total Profit %"
]

for c in numCols:
    prodItems[c] = (
        prodItems[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.replace(r"[^0-9\.\-]", "", regex=True)
    )
    prodItems[c] = pd.to_numeric(prodItems[c], errors="coerce")

In [27]:
prodItems["RevenueFixed"] = prodItems["Total Cost"].fillna(0) + prodItems["Total Profit"].fillna(0)

prodItems["ProfitMargin"] = np.where(
    prodItems["RevenueFixed"] > 0,
    prodItems["Total Profit"] / prodItems["RevenueFixed"],
    np.nan
)

prodItems[["Product Desc","Qty","Total Price","RevenueFixed","Total Cost","Total Profit","ProfitMargin"]].head(10)

,Product Desc,Qty,Total Price,RevenueFixed,Total Cost,Total Profit,ProfitMargin
8,1 SHOT DECAFE,404.00,0.00,0.00,"3,856.85","-3,856.85",NaN
9,2 SHOT DECAFE,637.00,0.00,0.00,"12,162.45","-12,162.45",NaN
10,3 SHOT DECAFE,113.00,0.00,0.00,"3,236.32","-3,236.32",NaN
11,ADD BANANA SAUCE LARGE,4.00,266.67,266.66,139.97,126.69,0.48
12,ADD BANANA SAUCE MEDIUM,47.00,"1,487.39","1,487.38",822.34,665.04,0.45
13,ADD BANANA SAUCE SMALL,28.00,893.72,893.73,489.91,403.82,0.45
14,ADD CARAMEL DRIZZLE,700.00,"23,370.36","23,370.37","2,349.38","21,020.99",0.90
15,ADD CARAMEL LARGE,313.00,"10,488.50","10,488.50","2,690.19","7,798.31",0.74
16,ADD CARAMEL MEDIUM,"1,142.00","37,584.33","37,584.33","8,923.01","28,661.32",0.76
17,ADD CARAMEL SF LARGE,236.00,"7,889.52","7,889.52","3,097.89","4,791.63",0.61


In [28]:
prodItems = prodItems.drop(columns=["Blank1","Blank2","Blank3"])
prodItems = prodItems.drop(columns=["Total Price"])
prodItems.shape
prodItems.columns


Index(['Product Desc', 'Qty', 'Total Cost', 'Total Cost %', 'Total Profit', 'Total Profit %', 'Branch', 'Service Type', 'Category', 'Section', 'RevenueFixed', 'ProfitMargin'], dtype='object')

In [29]:
# Load the dataset
df = pd.read_csv("rep_s_00673_SMRY.csv", header=None, dtype=str)

# Display the first few rows of the dataset to check its structure
# df.head(10)

# Remove the first row
headerIdx = df.index[
    df.apply(lambda r: r.astype(str).str.contains(r"\bCategory\b", na=False).any(), axis=1)
][0]
#slice from header row down, set proper headers
df_cleaned = df.iloc[headerIdx:].copy()

df_cleaned.columns = (
    df_cleaned.iloc[0]
    .astype(str)
    .str.strip()
    .str.replace("\n", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
)

df_cleaned = df_cleaned.iloc[1:].reset_index(drop=True)
df_cleaned.columns = (
    pd.Index(df_cleaned.columns)
      .astype(str)
      .str.strip()
      .str.replace("\n", " ", regex=False)
      .str.replace(r"\s+", " ", regex=True)
)
df_cleaned = df_cleaned.rename(columns={df_cleaned.columns[0]: "Category"})
df_cleaned = df_cleaned.loc[:, ~df_cleaned.columns.duplicated()]
df_cleaned = df_cleaned.loc[:, df_cleaned.columns.notna()]

# Remove rows containing the string "22-Jan-26 Years:2025 Month:0 Page 1 of 3"
df_cleaned = df_cleaned[~df_cleaned.astype(str).apply(lambda x: x.str.contains('Page', na=False)).any(axis=1)]

# Remove columns where all values are NaN
df_cleaned = df_cleaned.dropna(axis=1, how='all')

# Rename the columns
expected_cols = ['Category', 'Qty', 'Total Price', 'Total Cost', 'Total Cost %', 'Total Profit', 'Total Profit %']
df_cleaned = df_cleaned.reindex(columns=expected_cols)

# Remove rows containing "Total By Branch"
df_cleaned = df_cleaned[~df_cleaned['Category'].astype(str).str.contains('Total By Branch', na=False, case=False)]

# Normalize branch names in the 'Category' column (assuming branches are in the 'Category' column)
df_cleaned['Category'] = df_cleaned['Category'].astype(str).str.strip()

# Create a new column 'Branch' where branch names are identified
df_cleaned['Branch'] = df_cleaned['Category'].where(df_cleaned['Category'].str.startswith('Stories', na=False))

# Forward fill the 'Branch' column to propagate the branch name to product categories
df_cleaned['Branch'] = df_cleaned['Branch'].ffill()

# Remove rows where 'Category' contains only the branch name (those rows without 'BEVERAGES' or 'FOOD')
df_cleaned = df_cleaned[~df_cleaned['Category'].str.startswith('Stories', na=False)]

#convert numeric columns to float64 safely
numCols = ['Qty', 'Total Price', 'Total Cost', 'Total Cost %', 'Total Profit', 'Total Profit %']
for c in numCols:
      df_cleaned[c] = (
          df_cleaned[c]
          .astype(str)
          .str.replace(",", "", regex=False)
          .str.replace(r"[^0-9\.\-]", "", regex=True)
      )
      df_cleaned[c] = pd.to_numeric(df_cleaned[c], errors="coerce")

df_cleaned["RevenueFixed"] = df_cleaned.get("Total Cost", 0).fillna(0) + df_cleaned.get("Total Profit", 0).fillna(0)

# Move the 'Branch' column to the first position
cols = ['Branch'] + [col for col in df_cleaned.columns if col != 'Branch']
df_cleaned = df_cleaned[cols]

In [30]:
df_cleaned.head()

1,Branch,Category,Qty,Total Price,Total Cost,Total Cost %,Total Profit,Total Profit %,RevenueFixed
0,NaN,22-Jan-26,NaN,NaN,NaN,NaN,NaN,NaN,0.00
1,NaN,Category,NaN,NaN,NaN,NaN,NaN,NaN,0.00
3,Stories - Bir Hasan,BEVERAGES,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,Stories - Bir Hasan,FOOD,NaN,NaN,NaN,NaN,NaN,NaN,0.00
7,Stories Centro Mall,BEVERAGES,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [31]:
import pandas as pd

# Load the dataset
file_path = '/content/rep_s_00673_SMRY.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to check its structure
# df.head(10)

# Remove the first row
df_cleaned = df.drop(index=[0,2])
# Remove rows containing the string "22-Jan-26 Years:2025 Month:0 Page 1 of 3"
df_cleaned = df_cleaned[~df_cleaned.astype(str).apply(lambda x: x.str.contains('Page', na=False)).any(axis=1)]

# Remove columns where all values are NaN
df_cleaned = df_cleaned.dropna(axis=1, how='all')

# Rename the columns
df_cleaned.columns = ['Category', 'Qty', 'Total Price', 'Total Cost', 'Total Cost %', 'Total Profit', 'Total Profit %']

# Remove rows containing "Total By Branch"
df_cleaned = df_cleaned[~df_cleaned['Category'].str.contains('Total By Branch', na=False)]

# Normalize branch names in the 'Category' column (assuming branches are in the 'Category' column)
df_cleaned['Category'] = df_cleaned['Category'].str.strip().str.title()

# Create a new column 'Branch' where branch names are identified
df_cleaned['Branch'] = df_cleaned['Category'].apply(lambda x: x if x.startswith('Stories') else None)

# Forward fill the 'Branch' column to propagate the branch name to product categories
df_cleaned['Branch'] = df_cleaned['Branch'].fillna(method='ffill')

def toNumber(x):
    if pd.isna(x):
        return np.nan

    s = str(x).replace(",", "").strip()
    s = re.sub(r"[^0-9\.\-]", "", s)

    # remove multiple dots like 2026..
    if s.count(".") > 1:
        parts = s.split(".")
        s = parts[0] + "." + "".join(parts[1:])

    if s in ["", ".", "-", "-.", ".-"]:
        return np.nan

    try:
        return float(s)
    except:
        return np.nan
# Remove rows where 'Category' contains only the branch name (those rows without 'BEVERAGES' or 'FOOD')
df_cleaned = df_cleaned[~df_cleaned['Category'].str.startswith('Stories', na=False)]

numCols = ['Qty', 'Total Price', 'Total Cost', 'Total Cost %', 'Total Profit', 'Total Profit %']

for c in numCols:
    if c in df_cleaned.columns:
        df_cleaned[c] = df_cleaned[c].map(toNumber)

# Move the 'Branch' column to the first position
cols = ['Branch'] + [col for col in df_cleaned.columns if col != 'Branch']
df_cleaned = df_cleaned[cols]

/tmp/ipython-input-1217815029.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned['Branch'] = df_cleaned['Branch'].fillna(method='ffill')


In [32]:
df_cleaned.head()

,Branch,Category,Qty,Total Price,Total Cost,Total Cost %,Total Profit,Total Profit %
4,Stories - Bir Hasan,Beverages,"137,572.10","1,841,385.84","4,143,855.17",22.50,"14,270,003.22",77.50
5,Stories - Bir Hasan,Food,"102,791.40","1,264,966.00","4,658,225.80",36.82,"7,991,434.23",63.18
8,Stories Centro Mall,Beverages,"138,680.50","2,054,988.70","4,591,734.79",22.34,"15,958,152.17",77.66
9,Stories Centro Mall,Food,"121,423.10","1,307,149.74","4,981,860.70",38.11,"8,089,636.74",61.89
12,Stories Lau,Beverages,"58,430.00","777,228.30","1,774,898.67",22.84,"5,997,384.33",77.16


In [33]:
# Load the new dataset
file_path_new = '/content/rep_s_00191_SMRY-3.csv'
sales = pd.read_csv(file_path_new)

# Remove the first row
sales_cleaned = sales.drop(index=[0, 2])

# Remove the last column
sales_cleaned = sales_cleaned.iloc[:, :-1]

# Remove rows that contain the word "Page"
sales_cleaned = sales_cleaned[~sales_cleaned.astype(str).apply(lambda x: x.str.contains('Page', na=False)).any(axis=1)]

# Rename the columns
sales_cleaned.columns = ['Description', 'Barcode', 'Qty', 'Total Amount']

# Remove the column that contains the 'Barcode' (assuming it's the column with 'Barcode' in the name)
sales_cleaned = sales_cleaned.loc[:, ~sales_cleaned.columns.str.contains('Barcode', case=False)]

# Remove rows where 'Description' column has values that act as headers (like 'Description', 'Qty', 'Total Amount')
sales_cleaned = sales_cleaned[~sales_cleaned['Description'].isin(['Description', 'Qty', 'Total Amount'])]

# Extract the group name from the 'Description' column
# Assuming the group name is in the format "Group: <GroupName>"
sales_cleaned['Group'] = sales_cleaned['Description'].apply(lambda x: x.split(":")[1].strip() if 'Group:' in str(x) else None)

# Fill the 'Group' column for the rows where it's missing (e.g., rows related to product details)
sales_cleaned['Group'] = sales_cleaned['Group'].fillna(method='ffill')

# Remove rows where 'Qty' and 'Total Amount' are NaN and 'Group' is populated
sales_cleaned = sales_cleaned[~((sales_cleaned['Qty'].isna()) & (sales_cleaned['Total Amount'].isna()) & sales_cleaned['Group'].notna())]

# Extract the division name from the 'Description' column
# Assuming the division name is in the format "Division: <DivisionName>"
sales_cleaned['Division'] = sales_cleaned['Description'].apply(lambda x: x.split(":")[1].strip() if 'Division:' in str(x) else None)

# Fill the 'Division' column for the rows where it's missing (e.g., rows related to product details)
sales_cleaned['Division'] = sales_cleaned['Division'].fillna(method='ffill')

# Remove rows where 'Description' contains only division names (i.e., 'Qty' and 'Total Amount' are NaN)
sales_cleaned = sales_cleaned[~((sales_cleaned['Qty'].isna()) & (sales_cleaned['Total Amount'].isna()) & sales_cleaned['Division'].notna())]

# Extract the branch name from the 'Description' column
# Assuming the branch name is in the format "Branch: <BranchName>"
sales_cleaned['Branch'] = sales_cleaned['Description'].apply(lambda x: x.split(":")[1].strip() if 'Branch:' in str(x) else None)

# Fill the 'Branch' column for the rows where it's missing (e.g., rows related to product details)
sales_cleaned['Branch'] = sales_cleaned['Branch'].fillna(method='ffill')

# Remove rows where 'Description' contains only branch names (i.e., 'Qty' and 'Total Amount' are NaN)
sales_cleaned = sales_cleaned[~((sales_cleaned['Qty'].isna()) & (sales_cleaned['Total Amount'].isna()) & sales_cleaned['Branch'].notna())]

# Remove rows that contain 'Total by Group' or 'Total by Division' in the 'Description' column
sales_cleaned = sales_cleaned[~sales_cleaned['Description'].str.contains('Total by', na=False)]

# Remove any non-numeric characters (e.g., commas, currency symbols) from 'Total Amount'
sales_cleaned['Total Amount'] = sales_cleaned['Total Amount'].replace({',': '', '€': '', '$': '', '£': ''}, regex=True)

# Convert 'Qty' and 'Total Amount' columns to float
sales_cleaned['Qty'] = pd.to_numeric(sales_cleaned['Qty'], errors='coerce')
sales_cleaned['Total Amount'] = pd.to_numeric(sales_cleaned['Total Amount'], errors='coerce')

# Display the first few rows of the new dataset
sales_cleaned.head(60)

/tmp/ipython-input-183179382.py:28: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sales_cleaned['Group'] = sales_cleaned['Group'].fillna(method='ffill')
/tmp/ipython-input-183179382.py:38: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sales_cleaned['Division'] = sales_cleaned['Division'].fillna(method='ffill')
/tmp/ipython-input-183179382.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sales_cleaned['Branch'] = sales_cleaned['Branch'].fillna(method='ffill')


,Description,Qty,Total Amount,Group,Division,Branch
6,ESPRESSO,"3,313.00","500,103.75",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
7,DOUBLE ESPRESSO,"3,200.00","498,631.33",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
8,AMERICANO SMALL,902.00,"176,816.53",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
9,AMERICANO MEDIUM,657.00,"152,807.36",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
10,AMERICANO LARGE,334.00,"88,443.43",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
11,BLACK COFFEE SMALL,823.00,"133,384.23",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
12,BLACK COFFEE MEDIUM,307.00,"62,723.78",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
13,BLACK COFFEE LARGE,86.00,"19,908.92",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
14,SINGLE ESPRESSO MACC,32.00,"5,156.93",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan
15,DOUBLE ESPRESSO MAC,35.00,"6,595.68",BLACK COFFEE,HOT BAR SECTION,Stories - Bir Hasan


In [34]:
df_cleaned.head()

,Branch,Category,Qty,Total Price,Total Cost,Total Cost %,Total Profit,Total Profit %
4,Stories - Bir Hasan,Beverages,"137,572.10","1,841,385.84","4,143,855.17",22.50,"14,270,003.22",77.50
5,Stories - Bir Hasan,Food,"102,791.40","1,264,966.00","4,658,225.80",36.82,"7,991,434.23",63.18
8,Stories Centro Mall,Beverages,"138,680.50","2,054,988.70","4,591,734.79",22.34,"15,958,152.17",77.66
9,Stories Centro Mall,Food,"121,423.10","1,307,149.74","4,981,860.70",38.11,"8,089,636.74",61.89
12,Stories Lau,Beverages,"58,430.00","777,228.30","1,774,898.67",22.84,"5,997,384.33",77.16


In [35]:
prodItems.head()

,Product Desc,Qty,Total Cost,Total Cost %,Total Profit,Total Profit %,Branch,Service Type,Category,Section,RevenueFixed,ProfitMargin
8,1 SHOT DECAFE,404.00,"3,856.85",0.00,"-3,856.85",100.00,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION,0.00,NaN
9,2 SHOT DECAFE,637.00,"12,162.45",0.00,"-12,162.45",100.00,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION,0.00,NaN
10,3 SHOT DECAFE,113.00,"3,236.32",0.00,"-3,236.32",100.00,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION,0.00,NaN
11,ADD BANANA SAUCE LARGE,4.00,139.97,52.49,126.69,47.51,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION,266.66,0.48
12,ADD BANANA SAUCE MEDIUM,47.00,822.34,55.29,665.04,44.71,Stories - Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION,"1,487.38",0.45


In [36]:
monthlyClean.head()

,Year,Branch Name,January,February,March,April,May,June,July,August,September,October,November,December,Annual Total
0,2025,Stories - Bir Hasan,"3,355,705.33","2,842,993.67","2,266,050.67","3,459,979.67","2,125,379.33","744,638.33","3,799,740.25","3,783,897.91","3,255,935.75","3,128,528.54","2,851,287.10","2,743,087.70","34,357,224.25"
1,2025,Stories Ain El Mreisseh,"12,648,546.67","9,929,973.33","9,849,926.67","13,025,923.33","7,208,975.00","2,883,403.33","11,889,329.16","11,610,097.92","11,021,123.75","10,516,783.81","9,387,782.61","9,640,996.43","119,612,862.01"
2,2025,Stories Airport,0.00,0.00,0.00,0.00,0.00,"8,428.83","2,940,003.01","6,963,426.45","7,467,197.62","8,414,672.70","7,029,720.14","6,604,828.86","39,428,277.61"
3,2025,Stories Antelias,"2,615,854.33","2,139,010.67","3,162,716.67","3,391,741.33","2,033,635.67","728,978.33","2,963,696.71","3,182,363.97","2,909,965.47","2,715,152.86","2,551,353.16","2,234,759.77","30,629,228.94"
4,2025,Stories Batroun,"4,266,517.33","3,388,117.33","4,890,198.33","5,516,881.67","2,963,980.33","1,311,240.00","6,135,003.02","6,751,770.29","5,041,150.76","4,372,225.54","4,272,667.88","5,163,712.03","54,073,464.51"
